In [1]:
import pandas as pd

from datetime import datetime

In [2]:
# Caricamento dei dataframe
df_articles = pd.read_csv('../data/daily_news.csv')
df_likes = pd.read_csv('../data/interactions.csv')

In [3]:
# Conteggio del numero di interazioni per ogni articolo
likes = df_articles.groupby('link')['likes'].count()

In [4]:
likes = df_articles['link'].to_frame().merge(likes, on='link')

In [5]:
df_articles['likes'] = df_articles['likes'].fillna(0)

In [6]:
df_articles['likes'].isnull().sum()

0

In [7]:
# Aggiornamento del dataframe con il numero di interazioni per ogni articolo
df_articles['likes'] = likes['likes']

In [8]:
# Unione dei due dataframe sulla colonna 'id'
df = pd.merge(df_articles, df_likes, on='link')
print(len(df))
df.head()

126


,date,title,image,link,text,summary,likes,timestamp,user_id,ip_address
0,2023-01-07,"Covid, cosa è Kraken, la nuova variante super-...",https://www.ilsole24ore.com/static/img/ilsole2...,https://www.ilsole24ore.com/art/cosa-e-kraken-...,I punti chiaveSei le versioni sotto la lenteSo...,NaN,3,2023-01-06 11:08:00,4fe596684dbd4518350ef949587308b0ce77ca47309529...,10.0.0.75
1,2023-01-07,"Covid, cosa è Kraken, la nuova variante super-...",https://www.ilsole24ore.com/static/img/ilsole2...,https://www.ilsole24ore.com/art/cosa-e-kraken-...,I punti chiaveSei le versioni sotto la lenteSo...,NaN,2,2023-01-06 11:08:00,4fe596684dbd4518350ef949587308b0ce77ca47309529...,10.0.0.75
2,2023-01-07,"Sicurezza stradale, Polstrada: 1.489 morti in ...",https://www.ilsole24ore.com/static/img/ilsole2...,https://www.ilsole24ore.com/art/sicurezza-stra...,I punti chiaveAumentano del 106 le persone fer...,NaN,2,2023-01-07 23:38:32,4ea54b2e2999e703c15fbd7e2712467eccec291010fc47...,10.0.0.75
3,2023-01-07,"Sicurezza stradale, Polstrada: 1.489 morti in ...",https://www.ilsole24ore.com/static/img/ilsole2...,https://www.ilsole24ore.com/art/sicurezza-stra...,I punti chiaveAumentano del 106 le persone fer...,NaN,4,2023-01-07 23:38:32,4ea54b2e2999e703c15fbd7e2712467eccec291010fc47...,10.0.0.75
4,2023-01-07,"Multe, entro il 31 gennaio la decisione dei co...",https://www.ilsole24ore.com/static/img/ilsole2...,https://www.ilsole24ore.com/art/multe-entro-31...,I punti chiaveStralcio possibile entro il 31 g...,NaN,2,2023-01-07 09:50:03,4ea54b2e2999e703c15fbd7e2712467eccec291010fc47...,10.0.0.75


In [9]:
# Creazione della matrice di utilizzo degli articoli
co_occurrence = df.pivot_table(index='link', columns='user_id', values='likes', fill_value=0)
co_occurrence.head()

user_id,18efbcda4e36b1114f95e51b8ccd83c1fb170b3d119792d3ccb37027e6857532,4ea54b2e2999e703c15fbd7e2712467eccec291010fc4735be00c4198b3232a6,4fe596684dbd4518350ef949587308b0ce77ca473095295c4138a287ff9fdf57
link,,,
https://forbes.it/2021/04/22/convenienti-ma-di-qualita-come-rendere-le-energie-rinnovabili-accessibili-a-tutti/,0.0,1.0,0.0
https://forbes.it/2022/10/17/arte-sport-innovazione-non-perderti-le-novita/,0.0,1.0,0.0
https://forbes.it/2022/10/18/top-100-manager-2022/,0.0,1.5,0.0
https://forbes.it/2023/01/02/da-stripe-a-spacex-le-ipo-tech-piu-attese-a-wall-street-nel-2023/,0.0,1.0,0.0
https://forbes.it/2023/01/03/spacex-raccoglie-750-milioni-di-dollari-di-nuovi-finanziamenti-ora-vale-137-miliardi-di-dollari/,1.0,1.0,0.0


In [11]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(co_occurrence.values)
distances, indices = knn.kneighbors(co_occurrence.values, n_neighbors=3)

In [ ]:
def recommend_movies(user, num_recommended_movies):

  print('The list of the Movies {} Has Watched \n'.format(user))

  for m in df[df[user] > 0][user].index.tolist():
    print(m)
  
  print('\n')

  recommended_movies = []

  for m in co_occurrence[co_occurrence[user] == 0].index.tolist():

    index_df = df.index.tolist().index(m)
    predicted_rating = df1.iloc[index_df, df1.columns.tolist().index(user)]
    recommended_movies.append((m, predicted_rating))

  sorted_rm = sorted(recommended_movies, key=lambda x:x[1], reverse=True)
  
  print('The list of the Recommended Movies \n')
  rank = 1
  for recommended_movie in sorted_rm[:num_recommended_movies]:
    
    print('{}: {} - predicted rating:{}'.format(rank, recommended_movie[0], recommended_movie[1]))
    rank = rank + 1

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(co_occurrence)

In [ ]:
links = df['link'].unique().tolist()

In [ ]:
def get_similar_items(link, similarities, links, df, date):
    link_index = links.index(link)
    if link_index >= len(links):
        return []
    similar_items = []
    for i, s in enumerate(similarities[link_index]):
        if s > 0:
            similar_items.append((links[i], s))
    similar_items = [item for item in similar_items if df[df['link'] == item[0]]['date'].iloc[0] == date]
    return sorted(similar_items, key=lambda x: x[1], reverse=True)



In [ ]:
def recommend_items(user_id, date, df, similarities, links):
    df_user = df[(df['user_id'] == user_id) & (df['date'] == date)]
    if df_user.empty:
        return "Non ci sono dati per l'utente selezionato oggi."
    if df_user['link'].iloc[0] not in links:
        return "Il link passato in input non esiste nel dataframe."
    co_occurrence = df_user.pivot_table(index='link', columns='user_id', values='likes', fill_value=0)
    sim = cosine_similarity(co_occurrence)
    similar_items = get_similar_items(df_user['link'].iloc[0], sim, links, df, date)
    if not similar_items:
        return "Non ci sono link simili a quello selezionato oggi."
    return similar_items

In [ ]:
today = datetime.today().strftime('%Y-%m-%d')

In [ ]:
# esempio di utilizzo
#recommendations = recommend_items('18efbcda4e36b1114f95e51b8ccd83c1fb170b3d119792d3ccb37027e6857532', df, similarities, links)
recommendations = recommend_items('18efbcda4e36b1114f95e51b8ccd83c1fb170b3d119792d3ccb37027e6857532', today, df, similarities, links)
print(recommendations)